In [1]:
from datetime import date, timedelta
from random import seed, randrange

Next, we'll create a lambda function to increment the effective date by a year. This is largely to make the code more readable.

In [2]:
increment_year = lambda date_in :date(date_in.year + 1, date_in.month, date_in.day)

Finally, we'll create a generator which gives us a randomized policy. There's not much special about the written premium. It's simply a number between 0 and one thousand.

Note 1) the use of our lambda function, 2) the use of a random number seed and 3) the `yield` keyword which makes this a generator.

In [3]:
def random_policy(new_seed = 1234):
  """
  This is a generator which produces a dictionary of basic policy information
  """
  seed(new_seed)
  while True:
    policy = {
      'written_premium': randrange(10**3),
      'effective_date': date(2020, 1, 1) + timedelta(randrange(365))
    }
    policy['expiration_date'] = increment_year(policy['effective_date'])
    yield policy

We can now generate a list of dictionaries by calling `next()` as many times as we like. We don't need the result of our call to `range()`. We indicate that we're ignoring it by assigning it to the `_` object.

In [4]:
policy_generator = random_policy()

policies = [next(policy_generator) for _ in range(10)]

Take a look at our policies:

In [5]:
policies[:2]

[{'written_premium': 989,
  'effective_date': datetime.date(2020, 8, 13),
  'expiration_date': datetime.date(2021, 8, 13)},
 {'written_premium': 119,
  'effective_date': datetime.date(2020, 1, 4),
  'expiration_date': datetime.date(2021, 1, 4)}]

And confirm that we have as many as we're expecting:

In [6]:
len(policies)

10

## Earned premium

Compose a function to give the earned premium based on two dates. For example, a policy written on July 1, 2020 is roughly 50% earned between July 1, 2020 and December 31, 2020.

In [7]:
def earned_premium(written_premium, policy_start, policy_end, earn_start, earn_end):
    """
    Calculate earned premium. Presumes an even (linear) earning pattern
    """
    policy_term = policy_end - policy_start
    earn_start = max(earn_start, policy_start)
    earn_end = min(earn_end, policy_end)
    frac_earned = (earn_end - earn_start) / (policy_end - policy_start)
    
    return written_premium * frac_earned

We can confirm that the function works with some sample data.

In [8]:
premium = 10**3
earned_premium(premium, date(2020, 7, 1), date(2021, 7, 1), date(2020, 1, 1), date(2021, 1, 1))
earned_premium(premium, date(2020, 7, 1), date(2021, 7, 1), date(2021, 1, 1), date(2022, 1, 1))

495.8904109589041

This function will return the calendar year intervals which bracket a list of dates. Note the use of `set()`.

In [9]:
def get_cy_intervals(dates_list):
    """
    This function will return a list of calendar years
    """
    the_years = set([a_date.year for a_date in dates_list])
    cy_year = [[date(a_year, 1, 1), date(a_year + 1, 1, 1)] for a_year in the_years]
    return cy_year

We can see it in action here:

In [10]:
some_dates = [
    date(2020, 7, 1)
    , date(2020, 12, 31)
    , date(2021, 6, 30)
]

get_cy_intervals(some_dates)

[[datetime.date(2020, 1, 1), datetime.date(2021, 1, 1)],
 [datetime.date(2021, 1, 1), datetime.date(2022, 1, 1)]]

Those two functions will let us compute the earned premium in all of the calendar years in a policy coverage period.

In [11]:
def earned_premium_cy(written_premium, policy_start, policy_end):
    """
    Calculate the amount of premium earned by calendar year for a single policy period
    """
    cy_return = {}
    for cy_start, cy_end in get_cy_intervals([policy_start, policy_end]):
        cy_return[cy_start.year] = earned_premium(
          written_premium, policy_start, policy_end, cy_start, cy_end
        )
    return cy_return


Does it work?

In [12]:
earned_premium_cy(10e3, date(2020, 7, 1), date(2021, 6, 30))

{2020: 5054.945054945055, 2021: 4945.054945054945}

It does!

## All together now

In [13]:
# Initialize empty dictionary to store total earned premium by calendar year
cy_earned = {}
for policy in policies:
  
  cy_ep = earned_premium_cy(policy['written_premium'], policy['effective_date'], policy['expiration_date'])
  
  for year in cy_ep:
    if year in cy_earned:
      cy_earned[year] += cy_ep[year]
    else:
      cy_earned[year] = cy_ep[year]

cy_earned

{2020: 1706.729598023804, 2021: 1602.270401976196}

Here's a sneak preview of what's happening next week:

In [14]:
import pandas as pd

pd.DataFrame(policies)

,written_premium,effective_date,expiration_date
0,989,2020-08-13,2021-08-13
1,119,2020-01-04,2021-01-04
2,92,2020-10-25,2021-10-25
3,35,2020-12-09,2021-12-09
4,709,2020-02-12,2021-02-12
5,100,2020-06-30,2021-06-30
6,242,2020-01-09,2021-01-09
7,31,2020-01-09,2021-01-09
8,354,2020-11-27,2021-11-27
9,638,2020-09-04,2021-09-04


For the student

1. Simulate 5,000 simulations from a Poisson with an expected value of 5
2. Do the same for a negative binomial. Use values of 0.2, 0.5 and 1.0 for the coefficient of variation